# Explanation of Units

The units for properties, pore-scale models and any variable used in OpenPNM are based on SI units. Therefore, it is important to assign values to the variables based on SI units. This notebook provides examples of some of the variables used in OpenPNM with their corresponding units and demonstrates the challenges related to units and unit conversions. As an example the units of some parameters used in OpenPNM are shown in following table:

| Parameter    | Unit    |
|---:| :-------------|
| network spacing | $m$       |
| pore.diameter | $m$       |
| pore.pressure | $Pa$       |
| pore.emperature | $K$     |
| pore.volume | $m^3$     |
| pore.viscosity | $Pa.s$  |
| pore.diffusivity | $m^2/s$  |

## Use of absolute pressure vs gauge pressure

To demonstrate the importance of defining parameter values with correct units, a pore-scale model is examined in this section.

In [2]:
import numpy as np
import openpnm as op
%config InlineBackend.figure_formats = ['svg']
np.random.seed(10)
%matplotlib inline
np.set_printoptions(precision=5)

In [8]:
pn = op.network.Cubic(shape=[15, 15, 15], spacing=1e-6)
pn.add_model_collection(op.models.collections.geometry.spheres_and_cylinders)
pn.regenerate_models()
phase = op.phase.Phase(network=pn)

The `Phase` class in op.phase creates a generic phase with default temperature=298 K and pressure=101325.0 Pa. We can now define a viscosity model to assign 'pore.viscosity' values:

In [18]:
phase.add_model(propname='pore.viscosity', model=op.models.phase.viscosity.water_correlation)
phase.regenerate_models()
mean_vis = np.mean(phase['pore.viscosity'])
print(f'mean of water viscosity is: {mean_vis*1000:.3f} mPa.s')

mean of water viscosity is: 0.893 mPa.s


Which matches with reported values of water viscosity at 298 K. Note that assigning the pressure values as gauge pressure  or assigning the temperature as Celsius results in an incorrect estimation of viscosity. The reason is the pore-scale model equations were defined based on absolute pressure and absolute temperature.

## Unit conversion packages and challenges

Different packages are available for unit conversions such as [unyt](https://unyt.readthedocs.io/en/stable/modules/unyt.html) and [Pint](https://pint.readthedocs.io/en/0.6/). Despite their broad applications and working with numpy, they were not included in the development of pore-scale models in OpenPNM, due to some challenges including additional overhead in calculations and regenerating models. However, these packages can be used to convert the units of a pre-defined array in a different unit to SI unit and using the resulting numpy array (SI units) for any purpose in pore network modeling.